# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 27 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# Libraries

In [1]:
!pip install torchsummaryX wandb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 9.2 MB/s eta 0:00:00


In [2]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [3]:
### If you are using colab, you can import google drive to save model checkpoints in a folder
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [5]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"jasongao0917","key":"9aaf0189605045a47217ffc7b87d06fc"}') 
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73274 sha256=b68a8b1180b03593be0029fceb9e227bab4464ef26e9b82403bba77bedd3f9a1
  Stored in directory: /root/.cache/pip/wheels/f3/67/7b/a6d668747974998471d29b230e7221dd01330ac34faebe4af4
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [6]:
# commands to download data from kaggle

!kaggle competitions download -c 11-785-s23-hw1p2
!mkdir '/content/data'

!unzip -qo '11-785-s23-hw1p2.zip' -d '/content/data'

100% 16.0G/16.0G [14:34<00:00, 20.1MB/s]
100% 16.0G/16.0G [14:34<00:00, 19.6MB/s]


# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

In [8]:
# Dataset class to load train and validation data

class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes = PHONEMES, context=0, partition = "train-clean-360"): # Feel free to add more arguments

        self.context = context
        self.phonemes = phonemes
        
        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir = root + partition + '/mfcc'
        # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data using root
        self.transcript_dir = root + partition + '/transcript'

        # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_names = sorted(os.listdir(self.mfcc_dir)) 
        # TODO: List files in self.transcript_dir using os.listdir in sorted order
        transcript_names = sorted(os.listdir(self.transcript_dir))

        # Making sure that we have the same no. of mfcc and transcripts
        assert len(mfcc_names) == len(transcript_names) 

        self.mfccs, self.transcripts = [], []

        # TODO: Iterate through mfccs and transcripts
        for i in range(len(mfcc_names)):
        #   Load a single mfcc
            mfcc = np.load(self.mfcc_dir + '/' + mfcc_names[i])
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc_mean = np.mean(mfcc, axis = 0)
            mfcc_std = np.std(mfcc, ddof = 1, axis = 0)
            mfcc = (mfcc - mfcc_mean)/mfcc_std
        #   Load the corresponding transcript
            transcript  = np.load(self.transcript_dir + '/' + transcript_names[i])
            transcript = transcript[1:-1] # Remove [SOS] and [EOS] from the transcript 
            # (Is there an efficient way to do this without traversing through the transcript?)
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)        

        # NOTE:
        # Each mfcc is of shape T1 x 27, T2 x 27, ...
        # Each transcript is of shape (T1+2) x 27, (T2+2) x 27 before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that 
        # the final shape is T x 27 (Where T = T1 + T2 + ...) 
        self.mfccs = np.concatenate(self.mfccs)

        # TODO: Concatenate all transcripts in self.transcripts such that 
        # the final shape is (T,) meaning, each time step has one phoneme output
        self.transcripts = np.concatenate(self.transcripts)
        # Hint: Use numpy to concatenate

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)

        # Take some time to think about what we have done. 
        # self.mfcc is an array of the format (Frames x Features). 
        # Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is. 
        # We can introduce context by padding zeros on top and bottom of self.mfcc
        self.mfccs = np.pad(self.mfccs, [(self.context, self.context), (0, 0)]) # TODO

        # The available phonemes in the transcript are of string data type
        # But the neural network cannot predict strings as such. 
        # Hence, we map these phonemes to integers

        # TODO: Map the phonemes to their corresponding list indexes in self.phonemes
        self.transcripts = np.array([self.phonemes.index(t) for t in self.transcripts]) 
        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        
        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        frames = self.mfccs[ind : ind + 1 + 2 * self.context, : ]
        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data

        frames = torch.FloatTensor(frames) # Convert to tensors
        phonemes = torch.tensor(self.transcripts[ind])       

        return frames, phonemes

In [9]:
# TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
# Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.
class AudioTestDataset(torch.utils.data.Dataset):
    def __init__(self, root, phonemes = PHONEMES, context=0, partition= "test-clean"):
      self.context = context
      self.phonemes = phonemes
        
      # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
      self.mfcc_dir = root + partition + '/mfcc'
      # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
      mfcc_names = sorted(os.listdir(self.mfcc_dir)) 
      
      self.mfccs = []
      # TODO: Iterate through mfccs and transcripts
      for i in range(len(mfcc_names)):
      #   Load a single mfcc
        mfcc = np.load(self.mfcc_dir + '/' + mfcc_names[i])
        #   Do Cepstral Normalization of mfcc (explained in writeup)
        mfcc_mean = np.mean(mfcc, axis = 0)
        mfcc_std = np.std(mfcc, ddof = 1, axis = 0)
        mfcc = (mfcc - mfcc_mean)/mfcc_std
        self.mfccs.append(mfcc)
      # TODO: Concatenate all mfccs in self.mfccs such that 
      # the final shape is T x 27 (Where T = T1 + T2 + ...) 
      self.mfccs = np.concatenate(self.mfccs)
      # Length of the dataset is now the length of concatenated mfccs/transcripts
      self.length = len(self.mfccs)
      self.mfccs = np.pad(self.mfccs, [(self.context, self.context), (0, 0)])

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        
        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        frames = self.mfccs[ind : ind + 1 + 2 * self.context, : ]
        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data
        frames = torch.FloatTensor(frames) # Convert to tensors
       
        return frames

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments. 

In [10]:
config = {
    'epochs'        : 18,
    'batch_size'    : 2048,
    'context'       : 25,
    'init_lr'       : 1e-3,
    'drop_out'      : 0.1,
    'architecture'  : 'medium-cutoff'
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

# Create Datasets

### Parameters Configuration for different stages of of the training process
<br>1st stage: train model with the "train-clean-360" dataset for 18 epochs with the following parameters:
<br>>batch size = 2048
<br>>context = 25
<br>>initial learning rate = 0.001
<br>>drop out rate = 0.1
<br>>Since a ReduceLROnPlateau scheduler with patience of 2 is used, the learning rate will decrease to 0.0001 at some point during the training process.
<br>>After the first stage of training, the model has the following metrics:
train accuracy=90.38, train loss=0.2583, validation accuracy=88.214, validation loss=0.3582
<br>[Important] Save this "360-18e" model by using torch.save(model.state_dict(),) to the drive.
*********************************************************************
<br>2nd stage: Start with "360-18e" model built previously, continue training model with the "train-clean-100" dataset for 2 epochs with the following parameters: 
<br>>batch size = 2048
<br>>context = 25
<br>>initial learning rate = 0.0001
<br>>drop out rate = 0.1
<br>>After the second stage of training, the model has the following metrics:
train accuracy=87.476, train loss=0.3588, validation accuracy=88.4, validation loss=0.3303
We are happy now because our validation accuracy has increased from 88.214 to 88.4
<br>[Important] Save this "360-18e-100-2e" model by using torch.save(model.state_dict(),) to the drive.
*********************************************************************
<br>3rd stage: Start with the "360-18e-100-2e" model built previously, continue training model with the "train-clean-100" dataset for 10 epochs with the following parameters:
<br>>batch size = 2048
<br>>context = 25
<br>>initial learning rate = 0.00005
<br>>drop out rate = 0.1
<br>>Since a ReduceLROnPlateau scheduler with patience of 2 is used, the learning rate will decrease to 5e-7 at some point during the training process.
<br>>After the third stage of training, the model has the following metrics:
train accuracy=88.361, train loss=0.3276, validation accuracy=88.452, validation loss=0.3301
<br>>Our validation accuracy has increased from 88.4 to 88.452

### Complete the training process and procede to prediction.
### As a result, the ultimate parameters are based on a model trained with 18 epochs of "train-clean-360" data and 12 epochs of "train-clean-100" with constant batch size of 2048, context of 25, drop out rate of 0.1, and various initial learning rates.

### To switch between training datasets, please change the value of partition located in the constructor of the AudioDataset class.

In [11]:
#TODO: Create a dataset object using the AudioDataset class for the training data 
train_data = AudioDataset(root = '/content/data/11-785-s23-hw1p2/', phonemes = PHONEMES[0:40], context = config['context'])

# TODO: Create a dataset object using the AudioDataset class for the validation data 
val_data = AudioDataset(root = '/content/data/11-785-s23-hw1p2/', phonemes = PHONEMES[0:40], context = config['context'], partition = 'dev-clean')

# TODO: Create a dataset object using the AudioTestDataset class for the test data 
test_data = AudioTestDataset(root = '/content/data/11-785-s23-hw1p2/', phonemes = PHONEMES[0:40], context = config['context'], partition = 'test-clean')

### Datasets are loaded using the provided dataloader codes. num_workers 

In [12]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?

train_loader = torch.utils.data.DataLoader(
    dataset     = train_data, 
    num_workers = 4,
    batch_size  = config['batch_size'], 
    pin_memory  = True,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data, 
    num_workers = 4,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data, 
    num_workers = 4, 
    batch_size  = config['batch_size'], 
    pin_memory  = True, 
    shuffle     = False
)


print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*27)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size     :  2048
Context        :  25
Input size     :  1377
Output symbols :  42
Train dataset samples = 130453995, batches = 63699
Validation dataset samples = 1928204, batches = 942
Test dataset samples = 1934138, batches = 945


In [13]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([2048, 1377]) torch.Size([2048])


frames.shape = torch.Size([2048, 1377])
<br>phoneme.shape = torch.Size([2048]

# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

### Experiments are run on the architecture of pyramidal models with decreasing width of hidden layers. The ultimate model is with 6 hidden layers. The first three all have equal width of 2048 inputs and 2048 outputs. The fourth hidden layer has 2048 inputs and 1024 outputs. The fifth hidden layer has 1024 inputs and 1024 outputs. And the sixth hidden layer has 1024 inputs and 512 outputs. Batchnorm and dropout is performed at each layer. GELU activation is performed at each layer. Specifically, all batchnorm processes are applied before activations at all layers.

In [14]:
# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff

class Network(torch.nn.Module):

    def __init__(self, input_size, output_size):

        super(Network, self).__init__()
        self.linear1 = torch.nn.Linear(input_size, 2048)
        self.linear2 = torch.nn.Linear(2048,2048)
        self.linear3 = torch.nn.Linear(2048,2048)
        self.linear4 = torch.nn.Linear(2048,2048)
        self.linear5 = torch.nn.Linear(2048,1024)
        self.linear6 = torch.nn.Linear(1024,1024)
        self.linear7 = torch.nn.Linear(1024,512)
        self.linear8 = torch.nn.Linear(512,output_size)

        self.batchnorm1 = torch.nn.BatchNorm1d(2048)
        self.batchnorm2 = torch.nn.BatchNorm1d(2048)
        self.batchnorm3 = torch.nn.BatchNorm1d(2048)
        self.batchnorm4 = torch.nn.BatchNorm1d(2048)
        self.batchnorm5 = torch.nn.BatchNorm1d(1024)
        self.batchnorm6 = torch.nn.BatchNorm1d(1024)
        self.batchnorm7 = torch.nn.BatchNorm1d(512)

        self.dropout1 = torch.nn.Dropout(p=config['drop_out'])
        self.dropout2 = torch.nn.Dropout(p=config['drop_out'])
        self.dropout3 = torch.nn.Dropout(p=config['drop_out'])
        self.dropout4 = torch.nn.Dropout(p=config['drop_out'])
        self.dropout5 = torch.nn.Dropout(p=config['drop_out'])
        self.dropout6 = torch.nn.Dropout(p=config['drop_out'])
        self.dropout7 = torch.nn.Dropout(p=config['drop_out'])

        self.activation = torch.nn.GELU()

    def forward(self, A):
      
        L1 = self.linear1(A)
        B1 = self.batchnorm1(L1)
        A1 = self.activation(B1)
        A1_ = self.dropout1(A1)

        L2 = self.linear2(A1_)
        B2 = self.batchnorm2(L2)
        A2 = self.activation(B2)
        A2_ = self.dropout2(A2)

        L3 = self.linear3(A2_)
        B3 = self.batchnorm3(L3)
        A3 = self.activation(B3)
        A3_ = self.dropout3(A3)

        L4 = self.linear4(A3_)
        B4 = self.batchnorm4(L4)
        A4 = self.activation(B4)
        A4_ = self.dropout4(A4)

        L5 = self.linear5(A4_)
        B5 = self.batchnorm5(L5)
        A5 = self.activation(B5)
        A5_ = self.dropout5(A5)

        L6 = self.linear6(A5_)
        B6 = self.batchnorm6(L6)
        A6 = self.activation(B6)
        A6_ = self.dropout6(A6)

        L7 = self.linear7(A6_)
        B7 = self.batchnorm7(L7)
        A7 = self.activation(B7)
        A7_ = self.dropout7(A7)

        result = self.linear8(A7_)

        return result

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler. 

In [15]:
INPUT_SIZE  = (2*config['context'] + 1) * 27 # Why is this the case?
model       = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)
summary(model, frames.to(device))
# Check number of parameters of your network
# Remember, you are limited to 20 million parameters for HW1 (including ensembles)

               Kernel Shape  Output Shape     Params  Mult-Adds
Layer                                                          
0_linear1      [1377, 2048]  [2048, 2048]  2.822144M  2.820096M
1_batchnorm1         [2048]  [2048, 2048]     4.096k     2.048k
2_activation              -  [2048, 2048]          -          -
3_dropout1                -  [2048, 2048]          -          -
4_linear2      [2048, 2048]  [2048, 2048]  4.196352M  4.194304M
5_batchnorm2         [2048]  [2048, 2048]     4.096k     2.048k
6_activation              -  [2048, 2048]          -          -
7_dropout2                -  [2048, 2048]          -          -
8_linear3      [2048, 2048]  [2048, 2048]  4.196352M  4.194304M
9_batchnorm3         [2048]  [2048, 2048]     4.096k     2.048k
10_activation             -  [2048, 2048]          -          -
11_dropout3               -  [2048, 2048]          -          -
12_linear4     [2048, 2048]  [2048, 2048]  4.196352M  4.194304M
13_batchnorm4        [2048]  [2048, 2048

/usr/local/lib/python3.8/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_linear1,"[1377, 2048]","[2048, 2048]",2822144.0,2820096.0
1_batchnorm1,[2048],"[2048, 2048]",4096.0,2048.0
2_activation,-,"[2048, 2048]",NaN,NaN
3_dropout1,-,"[2048, 2048]",NaN,NaN
4_linear2,"[2048, 2048]","[2048, 2048]",4196352.0,4194304.0
5_batchnorm2,[2048],"[2048, 2048]",4096.0,2048.0
6_activation,-,"[2048, 2048]",NaN,NaN
7_dropout2,-,"[2048, 2048]",NaN,NaN
8_linear3,"[2048, 2048]","[2048, 2048]",4196352.0,4194304.0


The total number parameters used in this model is 19.1258M, which does not exceed the 20M upper limit.

### The loss function used in this training will cross entropy loss. The optimizer we select is Adam. And the learning rate scheduler used is ReduceLROnPlateau with patience equal to 2, meaning that we will change the current learning rate to its 1/10 if the loss has not been improving in the past 2 epochs.

In [16]:
criterion = torch.nn.CrossEntropyLoss() # Defining Loss function. 
# We use CE because the task is multi-class classification 

optimizer = torch.optim.Adam(model.parameters(), lr = config['init_lr'],) #Defining Optimizer
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience = 2, verbose = True)
# Recommended : Define Scheduler for Learning Rate, 
# including but not limited to StepLR, MultiStepLR, CosineAnnealingLR, ReduceLROnPlateau, etc. 
# You can refer to Pytorch documentation for more information on how to use them.

# Is your training time very high? 
# Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it 
# Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [17]:
torch.cuda.empty_cache()
gc.collect()

407

### Mixed precision training is utilized so that we can speed up the training process a bit.

In [18]:
scaler = torch.cuda.amp.GradScaler(enabled=True)

In [19]:
def train(model, dataloader, optimizer, criterion):

    model.train()
    tloss, tacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')
    
    for i, (frames, phonemes) in enumerate(dataloader):
        
        ### Initialize Gradients
        optimizer.zero_grad()

        ### Move Data to Device (Ideally GPU)
        frames = frames.to(device)
        phonemes = phonemes.to(device)

        with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=True):
          ### Forward Propagation
          logits  = model(frames)

          ### Loss Calculation
          loss = criterion(logits, phonemes)
        
        ### Gradient Descent
        # mixed precision training is utilized so that we can speed up the training process
        # refered to https://pytorch.org/docs/stable/notes/amp_examples.html
        # calls backward() on scaled loss to create scaled gradients.
        scaler.scale(loss).backward()
        # scaler.step() first unscales the gradients of the optimizer's assigned params.
        scaler.step(optimizer)
        # updates the scale for next iteration.
        scaler.update()     


        tloss   += loss.item()
        tacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))), 
                              acc="{:.04f}%".format(float(tacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()
  
    batch_bar.close()
    tloss   /= len(train_loader)
    tacc    /= len(train_loader)

    return tloss, tacc

In [20]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode
    vloss, vacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Move data to device (ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode(): 
            ### Forward Propagation
            logits  = model(frames)
            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        vloss   += loss.item()
        vacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]
        
        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))), 
                              acc="{:.04f}%".format(float(vacc*100 / (i + 1))))
        batch_bar.update()
    
        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss   /= len(val_loader)
    vacc    /= len(val_loader)

    return vloss, vacc

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb. 

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [ ]:
wandb.login(key="06c8f81427188d28de31ac7bfe0ddeadd005abc3") #API Key is in your wandb account, under settings (wandb.ai/settings)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Create your wandb run
run = wandb.init(
    name    = "first-run", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    #id     = "y28t31uz", ### Insert specific run id here if you want to resume a previous run
    #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw1p2", ### Project should be created in your wandb account 
    config  = config ### Wandb Config for your run
)

wandb: Currently logged in as: jiayang2 (jgao). Use `wandb login --relogin` to force relogin


In [ ]:
### Save your model architecture as a string with str(model) 
model_arch  = str(model)

### Save it in a txt file 
arch_file   = open("model_arch.txt", "w")
file_write  = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

['/content/wandb/run-20230226_191948-hh30i00a/files/model_arch.txt']

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [ ]:
# load the model previously saved
# model.load_state_dict(torch.load("/content/360-18-100-2"))

<All keys matched successfully>

In [ ]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()
gc.collect()
wandb.watch(model, log="all")

for epoch in range(config['epochs']):

    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc       = eval(model, val_loader)

    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))
    # apply scheduler to validation loss to reduce the current learning rate to its one-tenth
    scheduler.step(val_loss)

    ### Log metrics at each epoch in your run 
    # Optionally, you can log at each batch inside train/eval functions 
    # (explore wandb documentation/wandb recitation)
    wandb.log({'train_acc': train_acc*100, 'train_loss': train_loss, 
               'val_acc': val_acc*100, 'valid_loss': val_loss, 'lr': curr_lr})

    ### Highly Recommended: Save checkpoint in drive and/or wandb if accuracy is better than your current best

### Finish your wandb run
run.finish()


Epoch 1/1


Train:   0%|          | 0/63699 [00:00<?, ?it/s]

Val:   0%|          | 0/942 [00:00<?, ?it/s]

	Train Acc 79.0153%	Train Loss 0.6402	 Learning Rate 0.0000500
	Val Acc 83.6844%	Val Loss 0.4797


lr,▁
train_acc,▁
train_loss,▁
val_acc,▁
valid_loss,▁
lr,5e-05
train_acc,79.01529
train_loss,0.64018
val_acc,83.68437
valid_loss,0.47965


In [ ]:
# save the current model to drive
# torch.save(model.state_dict(), "360-18-100-4")

# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *sample_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [ ]:
def test(model, test_loader):
    ### What you call for model to perform inference?
    
    model.eval() # TODO train or eval?

    ### List to store predicted phonemes of test data
    test_predictions = []

    ### Which mode do you need to avoid gradients?
    with torch.inference_mode(): # TODO

        for i, mfccs in enumerate(tqdm(test_loader)):

            mfccs   = mfccs.to(device)             
            
            logits  = model(mfccs)

            ### Get most likely predicted phoneme with argmax
            predicted_phonemes = torch.argmax(logits, dim=1)
    
            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval 
            # TODO
            test_predictions += predicted_phonemes.tolist()
     
            del mfccs, logits

    return test_predictions

Erros or warnings may pop up when calling the test function, just rerun the cell belowuntil it is successful.

In [ ]:
prediction = test(model, test_loader)

  0%|          | 0/945 [00:01<?, ?it/s]

Exception ignored in: Exception ignored in: Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f6fa6bf2160><function _MultiProcessingDataLoaderIter.__del__ at 0x7f6fa6bf2160><function _MultiProcessingDataLoaderIter.__del__ at 0x7f6fa6bf2160><function _MultiProcessingDataLoaderIter.__del__ at 0x7f6fa6bf2160>



Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1466, in __del__
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1466, in __del__
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1466, in __del__
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1466, in __del__
                self._shutdown_workers()self._shutdown_workers()self._shutdown_workers()


s

In [ ]:
phonemes = PHONEMES[0:40]
### Create CSV file with predictions
with open("./submission.csv", "w+") as f:
    f.write("id,labe  l\n")
    for i in range(len(prediction)):
        f.write("{},{}\n".format(i, phonemes[prediction[i]]))

In [ ]:
### Submit to kaggle competition using kaggle API (Uncomment below to use)
!kaggle competitions submit -c 11-785-s23-hw1p2 -f ./submission.csv -m "Test Submission"

### However, its always safer to download the csv file and then upload to kaggle

100% 19.3M/19.3M [00:05<00:00, 3.81MB/s]
Successfully submitted to Frame-Level Speech Recognition